In [ ]:
import whisperx

In [ ]:
from os import getenv
from torch import cuda
WHISPER_BATCH_SIZE = 2
LANGUAGE = "de"
asr_options = {
            "suppress_blank": getenv("WHISPER_SUPPRESS_BLANK", "True").lower() == "true",
            "temperatures": [float(getenv("WHISPER_TEMPERATURE", 0))],
            "log_prob_threshold": float(getenv("WHISPER_LOGPROB_THRESHOLD", -0.8)),
            "no_speech_threshold": float(getenv("WHISPER_NO_SPEECH_THRESHOLD", 0.4)),
            "beam_size": int(getenv("WHISPER_BEAM_SIZE", 5)),
            "repetition_penalty": float(getenv("WHISPER_REPETITION_PENALTY", 1)),
            "prompt_reset_on_temperature": float(getenv("WHISPER_PROMPT_RESET_ON_TEMPERATURE", 0.5)),
            "no_repeat_ngram_size": int(getenv("WHISPER_NO_REPEAT_NGRAM_SIZE", 0)),
            "condition_on_previous_text": getenv("WHISPER_CONDITION_ON_PREVIOUS_TEXT", "False").lower() == "true",
        }

In [ ]:
model_string = "large-v2"
download_root = "../../speechtotext/poc_server/whisper/whisperx"
model = whisperx.load_model(
            model_string, 
            asr_options=asr_options,
            device="cuda" if cuda.is_available() else "cpu", 
            compute_type="int8" if cuda.is_available() else "auto",
            download_root=download_root,
            return_scores=True,)

In [ ]:
import faster_whisper
model.tokenizer = faster_whisper.tokenizer.Tokenizer(model.model.hf_tokenizer,
                                                                    model.model.model.is_multilingual, 
                                                                    task="transcribe",
                                                                    language=LANGUAGE
                                                                    )

In [ ]:
import soundfile as sf
from IPython.display import Audio
test_file = "/Users/jklaff/db_dumps/1_untertitel-von-stephanie-geiges.wav"
speech, sr = sf.read(test_file)
Audio(speech, rate=sr)

In [ ]:
inputs = [{"inputs": speech.astype("float32")}]
outputs = model(inputs, batch_size=WHISPER_BATCH_SIZE, language=LANGUAGE)
outputs

In [ ]:
test_list = [
    (1,2,3),
    (1,2,3),
    (1,2,3),
    (1,2,3),
    
]
a_list, b_list, c_list = [list(x) for x in zip(*test_list)]

In [ ]:
b_list